In [1]:
# Dependencies
from splinter import Browser
from requests import get
import re
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import select
import urllib.request
from bs4 import BeautifulSoup
import requests
import pymongo

In [2]:
# r = requests.get(url)
# soup = BeautifulSoup(r.text, 'html.parser')
# print(soup)
executable_path = {"executable_path": "chromedriver.exe"}
url = 'https://twitchtracker.com/channels/viewership'

In [3]:
driver = webdriver.Chrome()
driver.get(url)

In [4]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
# print(soup)

In [5]:
streamer_table = soup.find_all(
                'table', 
                {'id': 'channels'})
print(streamer_table)

[<table class="table table-condensed text-center" id="channels">
<thead>
<tr>
<th></th>
<th></th>
<th></th>
<th class="sortable active" data-original-title="Last 30 days" data-toggle="tooltip" title="" width="8%"><a class="color-viewers" href="/channels/viewership">AVG Viewers</a></th>
<th data-original-title="Last 30 days" data-toggle="tooltip" title="" width="8%"><span class="color-streamed">Time Streamed</span></th>
<th class="sortable" data-original-title="Overall" data-toggle="tooltip" title="" width="10%"><a class="color-viewersMax" href="/channels/peak-viewers">All Time Peak Viewers</a></th>
<th class="sortable" data-original-title="Last 30 days" data-toggle="tooltip" title="" width="9%"><a class="color-watched" href="/channels/hours-watched">Hours Watched</a></th>
<th class="sortable" data-original-title="Last 30 days" data-toggle="tooltip" title="" width="7%"><i class="fas fa-crown color-gold"></i><br/><a class="color-gold" href="/channels/ranking">Rank</a></th>
<th class="sor

In [6]:
# Empty lists to store data
streamer_handle = []
avg_views = []
time_streamed = []
peak_viewers= []
hours_watched = []
rank_value = []
total_views = []

In [7]:
# Parse through each tag and pull out text info
channel = soup.find_all('a', {'style' : 'color:inherit'})
views = soup.find_all('td', {'class' : 'color-viewers'})
time = soup.find_all('td', {'class' : 'color-streamed'})
peak_views = soup.find_all('td', {'class' : 'color-viewersMax'})
hours = soup.find_all('td', {'class' : 'color-watched'})
rank = soup.find_all('span', {'class' : 'rank-value'})
all_views = soup.find_all('td', {'class' : 'color-views'})

In [8]:
for streamer in rank:
    rank_value.append(streamer.get_text())
print(rank_value)

['1', '2', '5', '9', '13', '8', '10', '6', '4', '3', '15', '11', '25', '17', '21', '16', '19', '12', '7', '31']


In [9]:
for streamer in channel:
    streamer_handle.append(streamer.get_text())
print(streamer_handle)

[]


In [10]:
for streamer in all_views:
    total_views.append(streamer.get_text())
print(total_views)

['31.2M', '100M', '13.9M', '9.25M', '8.45M', '136M', '61.8M', '141M', '108M', '267M', '92.8M', '219M', '9.38M', '100M', '22.0M', '24.2M', '115M', '434M', '348M', '188M']


In [11]:
for streamer in views:
    avg_views.append(streamer.get_text())
print(avg_views)


['256,800', '168,547', '122,926', '110,698', '93,375', '89,529', '88,628', '84,508', '78,109', '76,890', '72,274', '70,379', '66,810', '61,207', '61,037', '60,521', '58,548', '55,749', '55,241', '51,173']


In [12]:
for streamer in time:
    time_streamed.append(streamer.get_text())

print(time_streamed)

['23.5hours', '83.4hours', '30hours', '21.7hours', '13.1hours', '84.7hours', '55.3hours', '114.9hours', '169.7hours', '326.8hours', '48.4hours', '88.1hours', '15.2hours', '83.1hours', '51.9hours', '91.4hours', '73.7hours', '164hours', '249.5hours', '49.4hours']


In [13]:
for streamer in peak_views:
    peak_viewers.append(streamer.get_text())
print(peak_viewers)

['650,237', '313,347', '232,299', '206,847', '140,629', '344,676', '208,744', '533,239', '2,470,347', '222,720', '400,312', '365,983', '215,211', '212,049', '235,473', '203,070', '400,429', '516,289', '247,432', '264,955']


In [14]:
for streamer in hours: 
    hours_watched.append(streamer.get_text())
print(hours_watched)

['6.03M', '14.1M', '3.69M', '2.40M', '1.22M', '7.58M', '4.90M', '9.71M', '13.3M', '25.1M', '3.50M', '6.20M', '1.02M', '5.08M', '3.16M', '5.53M', '4.31M', '9.14M', '13.8M', '2.53M']


In [15]:
tracker_df = {'streamer_handle' : streamer_handle,
              'average_viewers':avg_views,
              'hrs_streamed': time_streamed,
              'all_time_peak': peak_viewers, 
              'hrs_watched': hours_watched,
              'rank' : rank_value, 
              'total_views_mil' : total_views}
# tracker_df


In [16]:
# Put the combined lists into a dataframe
complete_tracker_df = pd.DataFrame.from_dict(tracker_df, orient='index')
complete_tracker_df = complete_tracker_df.transpose()
# complete_tracker_df

In [17]:
# Clean up columns
complete_tracker_df['hrs_streamed'] = complete_tracker_df['hrs_streamed'].str.replace(r'hours', '')
complete_tracker_df['hrs_watched'] = complete_tracker_df['hrs_watched'].str.replace(r'M', '')
complete_tracker_df['total_views_mil'] = complete_tracker_df['total_views_mil'].str.replace(r'M', '')
complete_tracker_df

,streamer_handle,average_viewers,hrs_streamed,all_time_peak,hrs_watched,rank,total_views_mil
0,None,"256,800",23.5,"650,237",6.03,1,31.2
1,None,"168,547",83.4,"313,347",14.1,2,100
2,None,"122,926",30,"232,299",3.69,5,13.9
3,None,"110,698",21.7,"206,847",2.40,9,9.25
4,None,"93,375",13.1,"140,629",1.22,13,8.45
5,None,"89,529",84.7,"344,676",7.58,8,136
6,None,"88,628",55.3,"208,744",4.90,10,61.8
7,None,"84,508",114.9,"533,239",9.71,6,141
8,None,"78,109",169.7,"2,470,347",13.3,4,108
9,None,"76,890",326.8,"222,720",25.1,3,267
